<a href="https://colab.research.google.com/github/Zathey92/TensorFlow-MNIST/blob/master/Mnist_Elio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Github** https://github.com/Zathey92/TensorFlow-MNIST

Antes que nada es necesario estar familiarizado con este entorno de trabajo. Se trata de un notebook de python y permite la ejecucción de código de forma interactiva. Ideal para hacer pruebas!

Para empezar debe estar conectado a un entorno de ejecucion, para ello debe estar identificado en google y aparecer como conectado en la barra superior derecha de este notebook. En caso de no estarlo, haga click sobre las opciones que aparecen al lado y seleccione *`Conectar a un entorno de ejecucion alojado.`*

El codigo se divide en celdas de un color gris para diferenciarlo del texto y cada celda se ejecuta por separado.
Para ejecutar la siguiente celda haga click sobre ella y vera un botton de **'play**', púlselo y vera como devuelve un `Hola deipe`. 



In [0]:
# <---- Botton a mi izquierda
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys, getopt

from tensorflow.examples.tutorials.mnist import input_data #Para la descarga de datos automatica
import tensorflow as tf

FLAGS = None
tf.logging.set_verbosity(tf.logging.ERROR)

print('Hola deipe')

Si ha visto el mensaje ya sabe como desenvolverse con el entorno y el siguiente paso será la creación de una red neuronal. Pero antes:
## ¿Que es una red neuronal?

Una red neuronal es un sistema de procesamiento de la información compuesto por nodos o neuronas interconectadas. Estas neuronas asignan un valor numérico o peso a sus entradas y producen una salida que permite resolver problemas de regresión (predecir una cantidad) y clasificación (predecir una clase).

## ¿Como funcionan?
Cuando las entradas a una neurona superan un umbral, esta se activa propagando su salida. Las coneciones de entrada que son fuertes aportan más que las débiles a esta activación. A continuación se describe el modelo matemático más simple que implementa el funcionamiento de una neurona biológica.

<img src="http://drive.google.com/uc?export=view&id=0B2_2YevJnwmmeGYyVi1kOGFBUDcwbFBHUnJHNW9HTzNHanhB" alt="Modelo de una neurona" width = 800>

Esta guapa la imagen sacada de mi TFG eh?

Siendo `xi` la entrada y los parámetros ajustables `wi` y `b`, tenemos que la función para la regresión lineal es:
`z=suma(wi*xi)+b`

Si utilizamos una función para la activación de la neurona que aplique un umbral sobre `z`, tal que `a = f(z)`, obtenemos la ecuación de un clasificador lineal:
 **a = f(suma(wi*xi) + b)**

La aparicion de b no es más que para desplazar la salida y evitar el valor 0 (Del cual la red no aprende).

Hay un ejecicio de calculo de los parametros internos w y b en el [doc.pdf](https://github.com/Zathey92/TensorFlow-MNIST/blob/master/doc.pdf) del github para entender con mayor detalle como puede esa funcion clasificar problemas binarios (Es un simple problema de rectas).

El conectar neuronas en capas permite funciones más complejas y resolver problemas no lineales  de gran complejidad (siempre que se usen funciones de activacion lineales)

<img src="http://drive.google.com/uc?export=view&id=0B2_2YevJnwmmOG8zRnh4RWd0MV9WLXdlOXZVM3pla1dFX0Rn" alt="Modelo de una neurona" width=450>

Las salidas pueden representar la clase que quieras. Lo importante es elegir una funcion de error que dependa del varlor esperado y el valor devuelto por la red. Una vez conocemos el error podemos minimizarlo utilizando **descenso de gradiente** y **propagación hacia atrás**. Este proceso de minimizar la funcion de error modifica los parámetros de la red para que en su siguiente intento el resultado sea más correcto. 
A esto se le conoce como **etapa de entrenamiento** y una vez entrenada, la red, será capaz de recibir datos de entrada que nunca ha visto dar resultados precisos. 

#Creacion de la red
La siguiente celda tiene las clases necesarias para construir el modelo de la red. En los comentarios se explica que hace cada cosa.
Recuerde ejecutarlo antes de pasar a la siguiente sección.

In [0]:
class Network:
  def __init__(self, input, output,hidden):
    
    #Input | Definimos la variable de entrada. Devuelve un tipo especial de puntero conocido como tensor y de tipo placeholder (toma valor durante la ejecución)
    #A efectos prácticos se entrena con multiple entradas a la red. Este número se conoce como **batch size**
    #La entrada tiene de primera dimension(batch) tamaño None. Esto quiere decir que durante tiempo de ejecucion el tamaño es variable.
    #La segunda dimension tiene de tamaño input y es el numero de neuronas de entrada que se define durante la creación del modelo
    self.x = tf.placeholder(tf.float32, [None, input]) 
    
    #Hidden | Definimos nuestra capa de neuronas intermedias
    self.Wh = tf.Variable(tf.random_normal([input, hidden])) # Los pesos Wi de los que hablabamos. Se inicializan random y tamaño de neuronas 
    self.bh = tf.Variable(tf.random_normal([hidden]))        # Los bias de los que hablabamos. Se inicializan random.
    self.h = tf.nn.sigmoid(tf.matmul(self.x, self.Wh) + self.bh) # Definimos la funcion del clasificador lineal f(Wi*xi+b) Notar que usamos sigmoid como funcion de activación. Probar con ReLu
    
    #Output | Definimos nuestra capa de salida
    #Dimension(Batch, clases). output=2 para problemas binarios (probabilidad de si y probabilidad de no)
    #(se puede reducir a 1 en caso de que probabilidad >50 sea si y <50 no)
    self.Wy = tf.Variable(tf.random_normal([hidden, output]))
    self.by = tf.Variable(tf.random_normal([output]))
    self.y = tf.matmul(self.h, self.Wy) + self.by #Función de salida
    self.output = tf.nn.softmax(self.y) #Softmax convierte la salida a una distribución probabilistica. La función de activación a la salida de una red siempre es diferente a las internas.
    
    #Salida deseada | Definimos la variable de salida esperada. Gracias a este valor podemos calcular el error de la red y entrenarla. No se utiliza en tiempo de ejecución si no se entrena.
    self.y_ = tf.placeholder(tf.float32, [None, output])
    #Funcion de calculo de error probabilistico sobre clases. Se le pasa la salida de la red y la salida de 
    self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.y)) #Calculo del error usando softmax, cross entropy y la media entre los batches.

    #Función contraria a la de error bastante simplista pero que permite conocer cuando la red es precisa.
    self.correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1)) #Se compara la salida activada y la que deberia ser y devuelve una lista de booleans [true,false,false,true]
    self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32)) #Se hace cast a una lista binarias Ej: [1,0,0,1] y se cualcula la media 1+0+0+1/4 = 0.5

    self.sess = tf.InteractiveSession()
    tf.global_variables_initializer().run() #iniciando variables y sesion
		
  #Funcion para entrenar la red pasandole los datos de entrada.	
  def train(self, v_train,iterations=1000, lr=0.01,batch_size=100,evaluate=False,target=2):
    #Declaramos el algoritmo de descenso sobre la función de error y la cantidad cambio o curva de aprendizaje lr
    #Un lr muy pequeño y la red tardará mucho en entrenar. Un valor grande y puede que el resultado no sea tan preciso
    #Existen otros optimizer de tensorflow que permiten decrecer el lr  a medida que pasa el entrenamiento. 
    #Adam es otro optimizador que tiene parametros de tipo momentum aprendibles por la red. en mi TFG lo explico con detalle por si le interesa a alguien.
    train_step = tf.train.GradientDescentOptimizer(lr).minimize(self.cross_entropy) #Parece magia pero son simples derivadas
    accuracy_result=0
    i=0
    #Entrenamos durante un numero fijo de veces o hasta que el resultado nos guste
    while i<iterations and accuracy_result<target:
      i+=1
      #En cada iteración utilizamos un numero batchSize de entradas y salidas deseadas
      batch_xs, batch_ys = v_train.next_batch(batch_size)
      #Llamamos a la sesion en C de tensorflow para el rapido montaje y ejecución de nuestro grafo(modelo)
      #Pedimos ejecutar el train_step y la precisión pasando los valores en los placeholder x e y_
      _,accuracy_result =self.sess.run([train_step,self.accuracy], feed_dict={self.x: batch_xs, self.y_: batch_ys}) 
      if evaluate:
        print('Precision: ',accuracy_result)
			
  #Función para comprobar una vez entranada la red su calidad con datos que nunca ha visto.   
  def test(self,test_xs,test_ys,evaluate=0):
    if evaluate==1:
      loss = tf.reduce_sum(tf.square(self.y_ - tf.nn.softmax(self.y)))
      return self.sess.run(loss, feed_dict={self.x: test_xs,self.y_: test_ys})/len(test_xs)
			
    #Testeamos el modelo
    return self.sess.run([self.accuracy, self.output], feed_dict={self.x: test_xs,self.y_: test_ys}) #lanzamos el modelo con los valores de placeholder para accuracy
 
print('¿Listo para la ecución?')

Ahora que tenemos nuestro modelo simple de clasificador con redes neuronales es hora de darle una tarea para aprender.
Para este ejemplo utilizamos el conocido mnist. Un conjunto de imágenes de carácteres numéricos escritos a mano de **28x28 px** lo que sería una entrada de **784**. 

Utilizamos un batch de **200** imágenes en cada iteración y no importa que se repitan las imagenes despues de varias iteraciones (pero tampoco deberíamos sobreentrenar la red)

El problemas que queremos hacer es clasificar imágenes de dígitos según su contenido. Si la salida de la primera neurona de la red se activa decimos que la entrada fuel el digito 0 y la siguiente el digito 1 y asi sucesivamente hasta tener **10 neuronas de salida**.

In [0]:
#Cada pixel de la imagen de entrada, blanco o negro (-1,1). 
#Puede ser cualquier rango pero si esta centrado en 0 suele ser mejor para el entrenamiento ya que la relación se mantiene pero la complejidad disminuye
#10 neuronas de salida. Clases (0,1,2,3,4,5,6,7,8,9)
inputs=784 
outputs=10
hidden=100 #Mas neuronas más memoria. Para la complejidad de este problema con 100 sobra.
learning=0.5
iterations = 5500
batch_size=200
m=False #Mensajes durante el entrenamiento default=False	 
mnist = input_data.read_data_sets("/tmp/tensorflow/mnist/input_data", one_hot=True) #descarga de datos automatica en /tmp
net= Network(inputs,outputs,hidden) #Creamos nuestro modelo.
print('Que no te preocupe el mensaje espera por un resultado')
print('Entrenando...')
net.train(mnist.train, iterations,learning,batch_size,m)#Entrenamos  #mnist.train tiene 55000 images(784px) and labels(10)
print('Testeando...')
test_result, output = net.test(mnist.test.images, mnist.test.labels) #Testeamos con datos diferentes. Durante este proceso la red no aprende.

print('Porcentaje de acierto Test: ' , round(test_result*100,2) , '%') #mnist.test.images [10000,784] mnist.test.labels [10000,10] 

Ahora solo queda la parte más divertida. Visualizar los datos de testeo. A continuación puede ejecutar la siguiente celda para mostrar un valor diferente en cada instante.

In [0]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

index = random.randrange(1000)
imgplot = plt.imshow(np.reshape(mnist.test.images[index], (-1, 28))) #Mostramos un valor de entrada de la red durante el tessteo
plt.show()

print('Resultado de la red: ',np.argmax(output[index]))#Mostramos la salida activada de la red para esa entrada

Este modelo es muy simple y puede ser mejorado. Si quiere seguir aprendiendo puede editar el código y hacer sus propias pruebas